# Multi-Server MCP Client with LlamaIndex and Ollama

This Jupyter notebook demonstrates connecting to **multiple MCP servers** (database + weather) using a local LLM via Ollama. The setup includes:

- **Database Server** on `http://127.0.0.1:8000/sse` (people database)
- **Weather Server** on `http://127.0.0.1:8001/sse` (US weather data)
- **Local LLM** via Ollama (llama3.2)

In [5]:
import nest_asyncio
nest_asyncio.apply()

## 2  Setup a local LLM

In [6]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

llm = Ollama(model="llama3.2", request_timeout=120.0)
Settings.llm = llm

## 3  Initialize the MCP client and build the agent
Point the client at your local MCP server’s **SSE endpoint** (default shown below), and list the available tools.

In [7]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

# Connect to database server (port 8000)
db_client = BasicMCPClient("http://127.0.0.1:8000/sse")
db_tools = McpToolSpec(client=db_client)

# Connect to weather server (port 8001)
weather_client = BasicMCPClient("http://127.0.0.1:8001/sse")
weather_tools = McpToolSpec(client=weather_client)

print("Connected to both servers!")

In [8]:
# Get tools from database server
db_tool_list = await db_tools.to_tool_list_async()
print("=== DATABASE TOOLS ===")
for tool in db_tool_list:
    print(f"{tool.metadata.name}: {tool.metadata.description[:100]}...")

print("\n=== WEATHER TOOLS ===")
# Get tools from weather server
weather_tool_list = await weather_tools.to_tool_list_async()
for tool in weather_tool_list:
    print(f"{tool.metadata.name}: {tool.metadata.description[:100]}...")

# Combine all tools
all_tools = db_tool_list + weather_tool_list
print(f"\nTotal tools available: {len(all_tools)}")

add_data Add new data to the people table using a SQL INSERT query.

    Args:
        query (str): SQL INSERT query following this format:
            INSERT INTO people (name, age, profession)
            VALUES ('John Doe', 30, 'Engineer')
        
    Schema:
        - name: Text field (required)
        - age: Integer field (required)
        - profession: Text field (required)
        Note: 'id' field is auto-generated
    
    Returns:
        bool: True if data was added successfully, False otherwise
    
    Example:
        >>> query = '''
        ... INSERT INTO people (name, age, profession)
        ... VALUES ('Alice Smith', 25, 'Developer')
        ... '''
        >>> add_data(query)
        True
    
read_data Read data from the people table using a SQL SELECT query.

    Args:
        query (str, optional): SQL SELECT query. Defaults to "SELECT * FROM people".
            Examples:
            - "SELECT * FROM people"
            - "SELECT name, age FROM people WHERE ag

## 3  Define the system prompt
This prompt steers the LLM when it needs to decide how and when to call tools.

In [9]:
SYSTEM_PROMPT = """\
You are an AI assistant with access to both weather information and database operations.

Available capabilities:
- Database operations: Add and read people data (name, age, profession)
- Weather information: Get weather alerts and forecasts for US locations

Tools available:
- add_data(query): Add people to database using SQL INSERT
- read_data(query): Query people database using SQL SELECT  
- get_alerts(state): Get weather alerts for US states (use 2-letter codes like "CA", "NY")
- get_forecast(latitude, longitude): Get weather forecast for coordinates

You can help with database management, weather queries, or both!
"""

## 4  Helper function: `get_agent()`
Creates a `FunctionAgent` wired up with the MCP tool list and your chosen LLM.

In [10]:
from llama_index.tools.mcp import McpToolSpec
from llama_index.core.agent.workflow import FunctionAgent

async def get_agent(all_tools_list):
    agent = FunctionAgent(
        name="Multi-Server Agent",
        description="An agent that can work with both weather data and database operations.",
        tools=all_tools_list,
        llm=llm,
        system_prompt=SYSTEM_PROMPT,
    )
    return agent

## 5  Helper function: `handle_user_message()`
Streams intermediate tool calls (for transparency) and returns the final response.

In [11]:
from llama_index.core.agent.workflow import (
    FunctionAgent, 
    ToolCallResult, 
    ToolCall)

from llama_index.core.workflow import Context

async def handle_user_message(
    message_content: str,
    agent: FunctionAgent,
    agent_context: Context,
    verbose: bool = False,
):
    handler = agent.run(message_content, ctx=agent_context)
    async for event in handler.stream_events():
        if verbose and type(event) == ToolCall:
            print(f"Calling tool {event.tool_name} with kwargs {event.tool_kwargs}")
        elif verbose and type(event) == ToolCallResult:
            print(f"Tool {event.tool_name} returned {event.tool_output}")

    response = await handler
    return str(response)

## 6  Initialize the MCP client and build the agent
Point the client at your local MCP server’s **SSE endpoint** (default shown below), build the agent, and setup agent context.

In [12]:
# Create the multi-server agent using all tools from both servers
agent = await get_agent(all_tools)

# Create the agent context
agent_context = Context(agent)

print(f"🤖 Multi-server agent created with {len(all_tools)} tools!")
print("Ready to handle both database and weather requests!")

In [ ]:
# Run the agent!
while True:
    user_input = input("Enter your message: ")
    if user_input == "exit":
        break
    print("User: ", user_input)
    response = await handle_user_message(user_input, agent, agent_context, verbose=True)
    print("Agent: ", response)

Enter your message:  add to db: Momo is a research engineer, she is 66 years old.
User:  add to db: Momo is a research engineer, she is 66 years old.
Calling tool add_data with kwargs {'query': 'INSERT INTO people (name, age, profession) VALUES ('}
Tool add_data returned meta=None content=[TextContent(type='text', text='false', annotations=None)] isError=False
Calling tool add_data with kwargs {'query': 'INSERT INTO people (name, age, profession) VALUES ('}
Tool add_data returned meta=None content=[TextContent(type='text', text='false', annotations=None)] isError=False
Agent:  "INSERT INTO people (name, age, profession) VALUES ('Momo', 66, 'research engineer')".format(meta=None, content=[TextContent(type='text', text='false', annotations=None)], isError=False)
Enter your message:  now retrieve all the data entries we saved in the db
User:  now retrieve all the data entries we saved in the db
Calling tool read_data with kwargs {'query': 'SELECT * FROM people'}
Tool read_data returned me